# Data Augmention

### Required Modules

In [3]:
import pandas as pd

## Converting Relative to Absolute Values
This process is important in understanding the eucledian distance and joint angles

In [6]:
h, w = 360, 640
kd = pd.read_csv('./data/interim/contemporary_001/landmarks_rel.csv')
kd.loc[:, kd.columns.str.endswith('_x')] *= w
kd.loc[:, kd.columns.str.endswith('_y')] *= h
kd.loc[:, kd.columns.str.endswith('_y')] = h - kd.loc[:, kd.columns.str.endswith('_y')]

z_cols = kd.filter(regex='_z$')
z_min = round(z_cols.min().min(),2)
z_max = round(z_cols.max().max(),2)
z_mean = round(z_cols.mean().mean())
print(f"Smallest z-value: {z_min}, Largest z-value: {z_max}, Average: {z_mean}")

Smallest z-value: -0.68, Largest z-value: 0.79, Average: 0


## Adding Joints

Currently, MediaPipe uses 24 landmarks to joint. We estimate extra joints for the purpose of understanding body alignment.

![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Torso Joint
The code calculates the torso joint positions based on the positions of the left and right shoulders and hips. The resulting data contains additional joint positions that can be used for further analysis and visualization.

In [7]:
kd.loc[:, 'torso_x'] = (kd['left_shoulder_x'] + kd['right_shoulder_x'] + kd['left_hip_x'] + kd['right_hip_x']) / 4
kd.loc[:, 'torso_y'] = (kd['left_shoulder_y'] + kd['right_shoulder_y'] + kd['left_hip_y'] + kd['right_hip_y']) / 4
kd.loc[:, 'torso_z'] = (kd['left_shoulder_z'] + kd['right_shoulder_z'] + kd['left_hip_z'] + kd['right_hip_z']) / 4

In [8]:
kd.loc[:, 'forehead_x'] = (kd['left_eye_outer_x'] + kd['right_eye_outer_x']) / 2
kd.loc[:, 'forehead_y'] = (kd['left_eye_outer_y'] + kd['right_eye_outer_y']) / 2
kd.loc[:, 'forehead_z'] = (kd['left_eye_outer_z'] + kd['right_eye_outer_z']) / 2

## Angles of Connected Joints

Here we will calculate the angles of connected joints which can be relevant information for extracting gestures - for example bending of knees.

In [ ]:

# Estimated torso position
df.loc[:, 'torso_x'] = (df['left_shoulder_x'] + df['right_shoulder_x'] + df['left_hip_x'] + df['right_hip_x']) / 4
df.loc[:, 'torso_y'] = (df['left_shoulder_y'] + df['right_shoulder_y'] + df['left_hip_y'] + df['right_hip_y']) / 4
df.loc[:, 'torso_z'] = (df['left_shoulder_z'] + df['right_shoulder_z'] + df['left_hip_z'] + df['right_hip_z']) / 4

# Eucleadian distance between right shoulder to hip
df.loc[:, 'dis_right_shoulder_to_hip'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_shoulder_to_hip'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between hip to knee
df.loc[:, 'dis_right_hip_to_knee'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_hip_to_knee'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between knee to ankle
df.loc[:, 'dis_right_knee_to_ankle'] = np.sqrt((df['right_knee_y'] - df['right_ankle_y'])**2)
df.loc[:, 'dis_left_knee_to_ankle'] = np.sqrt((df['left_knee_y'] - df['left_ankle_y'])**2)

# Eucleadian distance between forehead to right shoulder
df.loc[:, 'dis_forehead_to_right_shoulder'] = np.sqrt((df['forehead_y'] - df['right_shoulder_y'])**2)
df.loc[:, 'dis_forehead_to_left_shoulder'] = np.sqrt((df['forehead_y'] - df['left_shoulder_y'])**2)

# Difference between height to the left and to the right
df.loc[:, 'total_height_r'] = df['dis_forehead_to_right_shoulder'] + df['dis_right_shoulder_to_hip'] + df['dis_right_hip_to_knee'] + df['dis_right_knee_to_ankle']
df.loc[:, 'total_height_l'] = df['dis_forehead_to_left_shoulder'] + df['dis_left_shoulder_to_hip'] + df['dis_left_hip_to_knee'] + df['dis_left_knee_to_ankle']
df.loc[:,'diff_height_r_and_l'] = (df['total_height_r'] - df['total_height_l']).abs()

# Difference in x-axis from hip to ankle
df.loc[:, 'diff_hip_to_ankle_r'] = (df['right_hip_x']-df['right_ankle_x']).abs()
df.loc[:, 'diff_hip_to_ankle_l'] = (df['left_hip_x']-df['left_ankle_x']).abs()

# calculate the angle between the hip and knee joint for the right leg
df['angle_hip_knee_r'] = df.apply(lambda row: math.degrees(math.atan2(row['right_knee_y'] - row['right_hip_y'], row['right_knee_x'] - row['right_hip_x'])), axis=1)
df['angle_hip_knee_l'] = df.apply(lambda row: math.degrees(math.atan2(row['left_knee_y'] - row['left_hip_y'], row['left_knee_x'] - row['left_hip_x'])), axis=1)

## Distance Between Joints
Here we will compute the distance between connected joints to descibe further upper and lower body movement.

## Summary

* In this notebook, we have identified the data augmention process using basic geometric methods.
* The limitation of using absolute values is.. and that video is assumed to be on the upright position (or where the performer is not inverted)